In [2]:
'''
Python 3.X
'''

import xml.etree.ElementTree as ET
import csv
import numpy as np


inFN = "framework.xdp"
outFN = "framework.csv"

tree = ET.parse(inFN)
root = tree.getroot()

'''
To do:
X values from drop-downs, radio button groups, ... (?)
    value:0 title:Blah Blah
    value:1 title:adsfasdfasd
X nothing under radio button groups
X indexes on full hierarchical path
X - tab/section names

For RPA 
- is there a way to use full pathname to retrieve fields
'''


def getNodeTag(nd):
    findClose = nd.tag.find("}")
    ndTag = nd.tag[findClose+1:]
    return ndTag

count=0
vList=None
tabNm=1
SectionNm="Header"

path = ""


        
# called at start of list
# builds a symbol table, counting occurrence of names
def prepForListNodes(nd):
    lvlDict1 = {}
    # pass 1
    for subNd in nd:
        if 'name' in subNd.attrib:
            name = subNd.attrib['name']
            if name in lvlDict1:
                cnt = lvlDict1[name]
                cnt = cnt+1
                lvlDict1[name] = cnt
            else:
                lvlDict1[name] = 0
            #print("saved in lvlDict1",name,lvlDict1)
    return lvlDict1

uniqueID = 1
def getHierName(nd,lvlDict1,lvlDict2):
    global uniqueID
    if 'name' in nd.attrib:
        name = nd.attrib['name']
        if name in lvlDict1:
            cnt = lvlDict1[name]  
            #print("collision")
            if cnt>0:    # are there more than one there?
                #print("multi",lvlDict2)
                if name in lvlDict2:
                    #print("got next",lvlDict2[name])
                    idx = lvlDict2[name]
                else:
                    #print("starting")
                    idx = 0
                lvlDict2[name] = idx+1  # set up for next one   
                name = name + "[" + str(idx) + "]"
        else:
            print("HUH???")
            print("name",name)
            print("lvlDict1",lvlDict1)
            
    else:
        #name = "nm-"+str(uniqueID)
        name="xyzzy"
        uniqueID = uniqueID+1
    return name,lvlDict2
    
def recurseNodeList(nd,pth,nLvl):
    lvlDict1 = prepForListNodes(nd)
    
    #print("lvlDict1",lvlDict1)
    lvlDict2 = {}
    
    for subNd in nd:
        name,lvlDict2 = getHierName(subNd,lvlDict1,lvlDict2)       
        # now recurse
        doSF(subNd,name,pth,nLvl+1 )
  
def recurseToFind(nd,tag):
    ndTag = getNodeTag(nd)
    if ndTag == tag:
        return nd
    for subNd in nd:
        retVal = recurseToFind(subNd,tag)
        if retVal is not None:
            return retVal
    return None
    
    
def extractChoiceList(nd,pth,nLvl):
    itemsNd = recurseToFind(nd,'items')
    if itemsNd is None:
        print("NO ITEMS!!!")
        return
    
    #print("extractChoiceList",nd)
    itemsNm = getNodeTag(itemsNd)
    #print("itemsNm",itemsNm)
    #print("extractChoiceList 2",itemsNd)
    # items / text
    iChoice = 1
    for subNd in itemsNd:
        ndTag = getNodeTag(subNd)
        if ndTag == "text":
            valTxt = str(subNd.text)
            #print("CHI+OICELIST",valTxt)
            cPath = pth+".ch-"+str(iChoice)
            dumpCSVLine(valTxt,"choice",cPath,nLvl+1)
            iChoice = iChoice+1
            
    
def extractRBNameValue(nd):
    ndTag = getNodeTag(nd)
    #print("extractRBNameValue",ndTag)
    captionNd = recurseToFind(nd,'caption')
    #print("caption",captionNd)
        
    valueNd = recurseToFind(captionNd,'value')
    captionNd = recurseToFind(valueNd,'text')
    exNd = recurseToFind(valueNd,'exData')
    captionTxt = "??"
    if captionNd is not None:
        captionTxt = captionNd.text
    elif exNd is not None:
        captionTxt = "<rich text not presented>"
        #bodyNd = recurseToFind(exNd,'body')
        #captionTxt = bodyNd.tail
        #print("EXDATA/BODY",captionTxt)
    else:
        print("YOWZE!!!!")
    
    captionTxt = captionTxt.replace("&amp;","&")
    
    #print("pre",captionTxt)
    #captionTxt = [str(r) for r in captionTxt]
    #print("post",captionTxt)
  
    itemsNd = recurseToFind(nd,'items')
    textNd = recurseToFind(itemsNd,'text')
    intNd = recurseToFind(itemsNd,'integer')
    valTxt = "?"
    if textNd is not None:
        valTxt = textNd.text
    elif intNd is not None:
        valTxt = intNd.text
    else:
        print("HUH HUH HUH HUH")
    #textTxt = recurseToFind(itemsNd,'text').text
    return valTxt,captionTxt

def getRBGroup(nd,pth,nLvl):
    #print("getRBGroup",nd,pth,nLvl)
    iRB = 1
    for subNd in nd:
        #print("subNd",subNd)
        #print("subtype",getNodeTag(subNd))
        ndTag = getNodeTag(subNd)
        if ndTag=="field":
            val,cap = extractRBNameValue(subNd)
            name = val+"/"+cap
            
            #print("RBGroup")
            #ame = subNd.attrib['name']
            
            rbPath = pth+".rb-"+str(iRB)
            iRB = iRB+1
            #print("RBG",rbPath)
            
            #print("RadioButton!",name)
            dumpCSVLine(name,"choice",rbPath,nLvl+1)    

def CSVreset():
    global vList,count,tabNm,SectionNm
    count=0
    tabNm=1
    SectionNm="Header"
    vList=[['#','tab','major','name','type','path']]
    

noWantTypes = ['SUBFORM','button','END-exclGroup','END-ROW','END-choiceList']
wantTypes = ['textEdit','numericEdit','checkButton','dateTimeEdit','exclGroup','choiceList','choice','TABLE','END-TABLE','ROW']
    
def dumpCSVLine(nm,ty,pth,nLvl):
    global vList,count,tabNm,SectionNm
    
    # remove comma from nm
    nm = nm.replace(",",";")
    
    # remove first 10 chars of path
    pth = pth[10:]
    
    if tabNm==1 and SectionNm=="Header" and nm=="geCB":
        SectionNm="Product Selection"   
    elif tabNm==1 and nm=="TabTwoContent":
        tabNm=2
        SectionNm="Submittal"
    elif tabNm==2 and nLvl==10 and ty=="SUBFORM":
        SectionNm=nm
        
    #if ty=="RadioButton":
    #    print("vList ADDING")
    
    if ty in wantTypes:
        count = count+1
        
        nm = str(nm.encode('ascii', 'ignore'))
        nm = nm[2:-1]

        vList.append([str(count),str(tabNm),SectionNm,nm,ty,pth])
    elif ty in noWantTypes:
        pass
    else:
        print("UNKNOWN TYPE",ty)



# nLvl - depth
# nCnt - count in a list
def doSF(nd, name, curPath, nLvl):
    #print("doSF",getNodeName(baseNd),curPath)

    if count>250000:
        return
    
    # prepare to recurse, as needed

    ndTag = getNodeTag(nd)
    thispath = curPath + name
    nextpath = thispath + "."
    
    #print("TAG is ",ndName)
    
    # switch on node type
    if ndTag == "subform":
        
        if 'columnWidths' in nd.attrib:
            #print("***TABLE***")
            dumpCSVLine(name,'TABLE',thispath,nLvl)
            recurseNodeList(nd,nextpath,nLvl)
            dumpCSVLine(name,'END-TABLE',thispath,nLvl)
        
        elif 'layout' in nd.attrib:
            sfLayout = nd.attrib['layout']
            if sfLayout=='row' and name=='Row':
                dumpCSVLine(name,'ROW',thispath,nLvl)
            if "tb" in sfLayout:
                dumpCSVLine(name,'SUBFORM',thispath,nLvl)

            recurseNodeList(nd,nextpath,nLvl)     # recurse
                
            if sfLayout=='row' and name=='Row':
                dumpCSVLine(name,'END-ROW',thispath,nLvl)
            
        else:  
            recurseNodeList(nd,nextpath,nLvl)     # recurse

    elif ndTag == "exclGroup":
        #print("***EXCLGROUP",name,thispath)
        dumpCSVLine(name,ndTag,thispath,nLvl)
        
        getRBGroup(nd,thispath,nLvl)
        
        #recurseNodeList(nd,nextpath,nLvl)     # recurse
        dumpCSVLine(name,"END-"+ndTag,thispath,nLvl)
        
    elif ndTag == "field":
        # type of field is under <ui>, e.g., TextEdit, choiceList, etc.
        uiNd = recurseToFind(nd,'ui')
        if uiNd is None:
            print("WHAT TYPE OF FIELD IS THIS???")
            return
        fldTypeNd = uiNd[0]
        fldTypeName = getNodeTag(fldTypeNd)
        dumpCSVLine(name,fldTypeName,thispath,nLvl)
        
        if fldTypeName=="choiceList":
            #print("extracting from choice list!")
            # collect the items/values
            extractChoiceList(nd,thispath,nLvl)
            dumpCSVLine(name,"END-"+fldTypeName,thispath,nLvl)
            
        
        '''
        #print("UI =",uiName)
        if uiName == "ui":
            typeNd = ui[0]
            typeNm = getNodeName(typeNd)
            #print("***FIELD",name,typeNm,curPath + name)
            dumpCSVLine(name,typeNm,thispath,nLvl)
            return
        else:
            print("UNKNOWN FIELD -ui-",curPath)
            return
        '''
    else:
            recurseNodeList(nd,nextpath,nLvl)     # recurse

        #name = member.find('name').text
        #print("sf",name)
        #return
    #print("done")
    
CSVreset()
doSF(root,"root","",0)
print('DONE',count)
npV = np.asarray(vList,dtype=np.str)
np.savetxt(outFN, npV, delimiter=",",fmt="%s")
print('File saved')
#vList

DONE 89
File saved


In [9]:
npV = np.asarray(vList,dtype=np.str)
#npV
npV.shape

(3183, 6)

In [10]:
npV.shape

(3183, 6)

In [11]:
#np.savetxt("fields.csv", npV, delimiter=",",fmt="%s")